In [ ]:
library(fdapace)
library(raster)

In [ ]:
#select the time series - change the TS according to the specral index that you want to evaluate
ts.file<- './Data/Time Series/TS_mcari.txt'
ts <- t(read.delim(ts.file,header = T, sep = " ", quote = "\"", dec = "."))

In [ ]:
#plotting all pixel time series
matplot(ts, type = 'l')

In [ ]:
#pixel coordinates
coord.file <- './Data/Time Series/Coord/coordinates_TS.txt'
pixel.coord <- read.delim(coord.file,header = T, sep = " ", quote = "\"", dec = ".")

In [ ]:
#preparing data as data.frame for FPCA
tp<- dim(ts)
tp1 <- tp[1]
tp2 <- tp[2]

dati<- as.vector(as.matrix((ts)))
time <- as.numeric(rep(1:tp1, tp2))
ID <- rep(1:tp2, each = tp1)
per.fpca <- as.data.frame(cbind(ID, dati, time))
# Turn the original data into a list of paired amplitude and timing lists
ts.f <- MakeFPCAInputs(per.fpca$ID, per.fpca$time, per.fpca$dati)

In [ ]:
#run FPCA 
fpcaObjTS <- FPCA(ts.f$Ly, ts.f$Lt,list(methodXi='IN', methodMuCovEst = 'smooth', userBwCov = 2, kernel= 'gauss',FVEthreshold=0.999))

In [ ]:
#some graphics 
plot(fpcaObjTS) #general results
CreateScreePlot(fpcaObjTS) #variation explained
#variation explainend  
round((fpcaObjTS$lambda/sum(fpcaObjTS$lambda))*100,3) #each component
round(cumsum(fpcaObjTS$lambda/sum(fpcaObjTS$lambda))*100,3) #cumulative

In [ ]:
#plot the modality of variation of the first component (change k for other components)
CreateModeOfVarPlot(fpcaObjTS,k = 1, rainbowPlot = TRUE,col= c( "blue","white","red"))# 

In [ ]:
#estract scores and joining with pixel.coordinates 
FPCA.scores <- fpcaObjTS$xiEst

FPCA.scores.coord <- cbind(pixel.coord,FPCA.scores)
FPCA.comp.rast <- rasterFromXYZ(FPCA.scores.coord) 

#plotting the spatial pattern of the FPCA components
plot(FPCA.comp.rast[[1]])


In [ ]:
#plotting temporal and spatial pattern of the selected components (useful for interpretation)
#change k for choose the component
k=1 #setting the first component
par(mfrow=c(1,2))
CreateModeOfVarPlot(fpcaObjTS,k = k, rainbowPlot = TRUE,col= c( "blue","white","red"))
plot(FPCA.comp.rast[[k]],col=colorRampPalette(c("blue", "white", "red"))(255))

In [21]:
#export predictor to be used for the classification stage...
#change the path according to the selected index...
crs(FPCA.comp.rast) <- "EPSG:32633"
writeRaster(FPCA.comp.rast, "./Data/Predictors/_tmpc_MCARI_FPCs.tif", datatype='FLT4S', overwrite=TRUE)